In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

#Define the stocks
tickers = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS']
#Define the time period
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=10*365)

#Fetch the data
stock_data = {}
for ticker in tickers:
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty:
            print(f"No data found for {ticker}.")
            continue
        stock_data[ticker] = data
        print(f"Successfully downloaded data for {ticker}")
    except Exception as e:
        print(f"Could not download data for {ticker}. Error: {e}")

C:\Users\rishabh\AppData\Local\Temp\ipykernel_34164\47100723.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\rishabh\AppData\Local\Temp\ipykernel_34164\47100723.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)


Successfully downloaded data for RELIANCE.NS


[*********************100%***********************]  1 of 1 completed
C:\Users\rishabh\AppData\Local\Temp\ipykernel_34164\47100723.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)


Successfully downloaded data for TCS.NS


Failed to get ticker 'HDFCBANK.NS' reason: Failed to perform, curl: (56) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  1 of 1 completed
C:\Users\rishabh\AppData\Local\Temp\ipykernel_34164\47100723.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)


Successfully downloaded data for HDFCBANK.NS


[*********************100%***********************]  1 of 1 completed
C:\Users\rishabh\AppData\Local\Temp\ipykernel_34164\47100723.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)


Successfully downloaded data for INFY.NS


[*********************100%***********************]  1 of 1 completed

Successfully downloaded data for ICICIBANK.NS

--- Sample Data for RELIANCE.NS ---
Price            Close        High         Low        Open      Volume
Ticker     RELIANCE.NS RELIANCE.NS RELIANCE.NS RELIANCE.NS RELIANCE.NS
Date                                                                  
2015-08-07  214.144699  216.244048  212.578330  212.980800     8608888
2015-08-10  210.783569  215.112794  209.826350  214.177344     9288894
2015-08-11  210.152649  211.816897  208.412246  211.240394     8762464
2015-08-12  205.127258  210.152650  204.616017  209.630532     9650325
2015-08-13  203.049667  207.531186  202.190343  206.019218     9340350
